In [170]:
from compas.datastructures import Mesh
from compas.datastructures import subdivision as sd
from compas_plotters import MeshPlotter
from compas.geometry import midpoint_point_point
import random
from statistics import mean

from utilities import draw_compas_mesh, export_obj_by_attribute
import mysubdivision as msd

def plot(m):
    plotter = MeshPlotter(m)
    plotter.draw_edges()
    plotter.draw_faces()
    plotter.show()

mesh = Mesh.from_json('ring_1.json')

draw_compas_mesh(mesh,size=10)

Mesh(color=array('white', dtype='<U5'), line_material=ShaderMaterial(), material=ShaderMaterial(side='DoubleSi…

In [171]:
# Start subdividing

mesh = Mesh.from_json('ring_1.json')

def subdivide_by_ftype(mesh):
    fkeys = list(mesh.faces())
    for fk in fkeys:
        location = mesh.get_face_attribute(fk, 'location')
    
        # OUTER RING ANTON
        if location == 'outer':
            new_keys = msd.taper_face(mesh, fk, height=.5, ratio=0.1)
            
            areas = []
            
            for new_key in new_keys:
                areas.append(mesh.face_area(new_key))
                
            for new_key in new_keys:
                if mesh.face_area(new_key) > mean(areas):
                    mesh.set_face_attribute(new_key, 'location', 'hub')
                else:
                    mesh.set_face_attribute(new_key, 'location', 'circulation')
        
        elif location == 'circulation':
            new_keys = msd.pyramid_face(mesh, fk, height=1)
            
            if random.random()<0.2:
                mesh.set_faces_attribute(new_keys, 'location', 'depot')
            else:
                mesh.set_faces_attribute(new_keys, 'location', 'conduits')
        
        elif location == 'conduits':
            new_key = msd.segment_face(mesh, fk, num=10, start_index=0)
            mesh.set_face_attribute(new_key, 'location', 'line')
             
        elif location == 'hub':
            new_keys = msd.segment_face(mesh, fk, num=10, start_index=0)
            mesh.set_faces_attribute(new_keys, 'location', 'part_hub')
        
        elif location == 'construction':
            pass
        
        # INNER RING MAHIRO
        elif location == 'inner':
            pass
        
        elif location == 'mountain':
            pass
        
        elif location == 'lake':
            pass
               
        # SIDES
        elif location == 'sides':
            pass
        
        else:
            pass

for _ in range(5):
    subdivide_by_ftype(mesh)

In [172]:
draw_compas_mesh(mesh)

export_obj_by_attribute('outer.obj', mesh, 'location')